In [1]:
.libPaths('/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library')

library(IRkernel)
library(jsonlite)
library(repr)

In [129]:
asd_sample_iso_IR <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/iso_retained_intron/asd_iso_IR.txt",header=F)
asd_sample_iso_no_IR <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/iso_retained_intron/asd_iso_no_IR.txt",header=F)

In [128]:
head(asd_sample_iso_IR)

,V1
,<chr>
1,ENST00000595014.1_3
2,ENST00000598345.1_3
3,ENST00000472360.1_3
4,ENST00000495442.1_3
5,ENST00000495709.1_3
6,ENST00000546562.6_4


In [5]:
#调取两组iso的exon边界，构建剪接供受体位点
exon_region <- read.csv("/disk1/wenqing/tmp_data/ASD/switchList_exon.csv")
dim(exon_region)

[1] 926332      9

In [52]:
iso_IR_exon_region <- exon_region[exon_region$isoform_id %in% asd_sample_iso_IR$V1,]
iso_no_IR_exon_region <- exon_region[exon_region$isoform_id %in% asd_sample_iso_no_IR$V1,]

In [126]:
head(iso_IR_exon_region)

,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
125,chr1,786728,787490,763,+,ENST00000425657.1_1,LINC01128,LINC01128
132,chr1,788051,788132,82,+,ENST00000425657.1_1,LINC01128,LINC01128
220,chr1,970247,970704,458,+,ENST00000469403.1_1,AGRN,AGRN
223,chr1,976045,976260,216,+,ENST00000469403.1_1,AGRN,AGRN
226,chr1,976553,976777,225,+,ENST00000469403.1_1,AGRN,AGRN
257,chr1,982581,982834,254,+,ENST00000466223.1_1,AGRN,AGRN


In [56]:
iso_IR_exon_region <- iso_IR_exon_region[,c(2:9)]
iso_no_IR_exon_region <- iso_no_IR_exon_region[,c(2:9)]
write.table(iso_IR_exon_region,"/disk1/wenqing/tmp_data/ASD/asd_iso_IR_exon_region.txt",row.names=F,quote=F)
write.table(iso_no_IR_exon_region,"/disk1/wenqing/tmp_data/ASD/asd_iso_no_IR_exon_region.txt",row.names=F,quote=F)

In [96]:
head(iso_IR_exon_region)

,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
125,chr1,786728,787490,763,+,ENST00000425657.1_1,LINC01128,LINC01128
132,chr1,788051,788132,82,+,ENST00000425657.1_1,LINC01128,LINC01128
220,chr1,970247,970704,458,+,ENST00000469403.1_1,AGRN,AGRN
223,chr1,976045,976260,216,+,ENST00000469403.1_1,AGRN,AGRN
226,chr1,976553,976777,225,+,ENST00000469403.1_1,AGRN,AGRN
257,chr1,982581,982834,254,+,ENST00000466223.1_1,AGRN,AGRN


In [99]:
start_exd <- iso_IR_exon_region$start-5
end_exd <- iso_IR_exon_region$end+5
iso_IR_exon_region_exd <- iso_IR_exon_region
iso_IR_exon_region_exd$start <- start_exd
iso_IR_exon_region_exd$end <- end_exd

In [100]:
no_IR_start_exd <- iso_no_IR_exon_region$start-5
no_IR_end_exd <- iso_no_IR_exon_region$end+5
iso_no_IR_exon_region_exd <- iso_no_IR_exon_region
iso_no_IR_exon_region_exd$start <- no_IR_start_exd
iso_no_IR_exon_region_exd$end <- no_IR_end_exd

In [104]:
write.table(iso_IR_exon_region_exd,"/disk1/wenqing/tmp_data/ASD/asd_iso_IR_exon_region_exd.txt",row.names=F,quote=F)
write.table(iso_no_IR_exon_region_exd,"/disk1/wenqing/tmp_data/ASD/asd_iso_no_IR_exon_region_exd.txt",row.names=F,quote=F)

In [106]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+5))
donor_sites_2_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-2))
donor_sites_1_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-1))
donor_sites_3_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-3))
donor_sites_4_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-4))
donor_sites_5_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-5))

acceptor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-5))

acceptor_sites_2_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+2))
acceptor_sites_1_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+1))
acceptor_sites_3_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+3))
acceptor_sites_4_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+4))
acceptor_sites_5_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5,
                        donor_sites_1_neg,donor_sites_2_neg,donor_sites_3_neg,donor_sites_4_neg,donor_sites_5_neg,
                        acceptor_sites_1_pos,acceptor_sites_2_pos,acceptor_sites_3_pos,acceptor_sites_4_pos,acceptor_sites_5_pos)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_IR_exon_region$seqnames)),
                        rep("donor_us_1",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_2",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_3",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_4",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_5",times=length(iso_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_IR_exon_region$seqnames)),
                        rep("acceptor_ds_1",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_2",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_3",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_4",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_5",times=length(iso_IR_exon_region$seqnames)))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)

isoform_id <- rep(iso_IR_exon_region$isoform_id,times=20)

In [107]:
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info,isoform_id))

In [130]:
head(all_sites)

,site,info,iso_id
,<chr>,<chr>,<chr>
1,chr1_12058,donor_ds_1_G,ENST00000450305.2_2
2,chr1_12228,donor_ds_1_G,ENST00000450305.2_2
3,chr1_12698,donor_ds_1_G,ENST00000450305.2_2
4,chr1_13053,donor_ds_1_G,ENST00000450305.2_2
5,chr1_13375,donor_ds_1_G,ENST00000450305.2_2
6,chr1_13671,donor_ds_1_G,ENST00000450305.2_2


In [110]:
dim(all_sites)
colnames(all_sites) <- c("site","info","iso_id")
head(all_sites)


[1] 2201380       3

,site,info,iso_id
,<chr>,<chr>,<chr>
1,chr1_787491,donor_ds_1_G,ENST00000425657.1_1
2,chr1_788133,donor_ds_1_G,ENST00000425657.1_1
3,chr1_970705,donor_ds_1_G,ENST00000469403.1_1
4,chr1_976261,donor_ds_1_G,ENST00000469403.1_1
5,chr1_976778,donor_ds_1_G,ENST00000469403.1_1
6,chr1_982835,donor_ds_1_G,ENST00000466223.1_1


In [111]:
uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site

[1] 1516131       3

In [40]:
#head(all_possible_sites)
#all.res.wizSampleInfo.new: 由ctr组中的23组成
all_res <- read.table("/disk1/wenqing/tmp_data/ASD/all.res.wizSampleInfo.new")

#asd only
all_res <- all_res[all_res$V9=='17'|all_res$V9=='18'|all_res$V9=='57',]
head(all_res)

all_res$V10 <- paste0(all_res$V1,"_",all_res$V3)
head(all_res)

,V1,V2,V3,V4,V5,V6,V7,V8,V9
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17,chr1_99235
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17,chr1_99299
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17,chr1_99313
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17,chr1_99327
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103106
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103107


In [105]:
write.table(all_res,"/disk1/wenqing/tmp_data/ASD/asd_male_pfc/asd_all.res",quote=F,col.names=F,row.names=F)

In [112]:
resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]


In [113]:
resInSpliceSites$V11 <- "info"
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info
resInSpliceSites$V12 <- "iso_id"
resInSpliceSites$V12 <- uniq_all_sites[resInSpliceSites$V10,]$iso_id



In [114]:
head(resInSpliceSites)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
4491,chr1,3762566,3762567,TC,1,+,AGGGAGTAGTTAGCGG,1:1,17,chr1_3762567,acceptor_us_2_A,ENST00000675966.1_1
4492,chr1,3762566,3762567,TC,2,+,TACGGATAGCCTTGAT,2:2,17,chr1_3762567,acceptor_us_2_A,ENST00000675966.1_1
4493,chr1,3762566,3762567,TC,2,-,TACAGTGTCACAAACC,1:1,17,chr1_3762567,acceptor_us_2_A,ENST00000675966.1_1
5859,chr1,6523677,6523678,TC,2,-,ACATACGGTGTGAATA,2:2,17,chr1_6523678,acceptor_us_4_N,ENST00000473343.5_3
6303,chr1,6726654,6726655,TC,2,+,GGCCGATTCAATCTCT,9:9,17,chr1_6726655,donor_us_2,ENST00000693337.1_1
28077,chr1,19543295,19543296,TC,1,+,TGGCCAGGTATCAGTC,2:2,17,chr1_19543296,donor_us_3,ENST00000690451.1_1


In [115]:
#计算as site有RES的iso数量，不去重
length(unique(resInSpliceSites$V10))

[1] 882

In [117]:
#发生在retained itron上的所有编辑位点
asd_res_in_iso_IR_region <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/iso_retained_intron/asd_res_in_iso_IR_region_exd.txt")$V1
head(asd_res_in_iso_IR_region)
length(setdiff(unique(asd_res_in_iso_IR_region),unique(resInSpliceSites$V10)))

[1] "chr1_983918"  "chr1_983922"  "chr1_1419591" "chr1_1419616" "chr1_1419625"
[6] "chr1_1419631"

In [84]:
length(unique(iso_IR_exon_region$isoform_id))

[1] 21416

In [85]:
length(unique(setdiff(iso_IR_exon_region$isoform_id,unique(resInSpliceSites$V12))))

[1] 20872

In [25]:
resInInIsoIR_region <- read.table("/disk1/wenqing/tmp_data/ASD/iso_IR_RES.txt")


[1] 192849      2

In [27]:
head(resInInIsoIR_region)


,V1,V2
,<chr>,<chr>
1,chr1_1554509,ENST00000477990.1_1
2,chr1_1554510,ENST00000477990.1_1
3,chr1_1554511,ENST00000477990.1_1
4,chr1_1554560,ENST00000477990.1_1
5,chr1_1554564,ENST00000477990.1_1
6,chr1_1554583,ENST00000477990.1_1


In [26]:
resNotInSpliceSites <- resInInIsoIR_region[!(resInInIsoIR_region$V1 %in% resInSpliceSites$V10),]
dim(resNotInSpliceSites)
length(unique(resNotInSpliceSites$V2))

[1] 192634      2

[1] 2133

In [ ]:
#下面为未发生IR的iso

In [119]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+5))
donor_sites_2_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-2))
donor_sites_1_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-1))
donor_sites_3_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-3))
donor_sites_4_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-4))
donor_sites_5_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-5))

acceptor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-5))

acceptor_sites_2_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+2))
acceptor_sites_1_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+1))
acceptor_sites_3_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+3))
acceptor_sites_4_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+4))
acceptor_sites_5_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5,
                        donor_sites_1_neg,donor_sites_2_neg,donor_sites_3_neg,donor_sites_4_neg,donor_sites_5_neg,
                        acceptor_sites_1_pos,acceptor_sites_2_pos,acceptor_sites_3_pos,acceptor_sites_4_pos,acceptor_sites_5_pos)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_no_IR_exon_region$seqnames)),
                        rep("donor_us_1",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_2",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_3",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_4",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_5",times=length(iso_no_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_no_IR_exon_region$seqnames)),
                        rep("acceptor_ds_1",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_2",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_3",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_4",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_5",times=length(iso_no_IR_exon_region$seqnames)))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)

isoform_id <- rep(iso_no_IR_exon_region$isoform_id,times=20)

all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info,isoform_id))

In [120]:
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info,isoform_id))

In [121]:

dim(all_sites)
colnames(all_sites) <- c("site","info","iso_id")
head(all_sites)

uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site

resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]

resInSpliceSites$V11 <- "info"
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info
resInSpliceSites$V12 <- "iso_id"
resInSpliceSites$V12 <- uniq_all_sites[resInSpliceSites$V10,]$iso_id

dim(resInSpliceSites)


[1] 14181580        3

,site,info,iso_id
,<chr>,<chr>,<chr>
1,chr1_12058,donor_ds_1_G,ENST00000450305.2_2
2,chr1_12228,donor_ds_1_G,ENST00000450305.2_2
3,chr1_12698,donor_ds_1_G,ENST00000450305.2_2
4,chr1_13053,donor_ds_1_G,ENST00000450305.2_2
5,chr1_13375,donor_ds_1_G,ENST00000450305.2_2
6,chr1_13671,donor_ds_1_G,ENST00000450305.2_2


[1] 6716037       3

[1] 13098    12

[1] 2685

In [122]:
length(unique(resInSpliceSites$V10))


[1] 4796

In [123]:
asd_res_in_iso_no_IR_region <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/iso_retained_intron/asd_res_in_iso_no_IR_region_exd.txt")$V1
head(asd_res_in_iso_no_IR_region)
length(setdiff(unique(asd_res_in_iso_no_IR_region),unique(resInSpliceSites$V10)))

[1] "chr1_134228" "chr1_134241" "chr1_134250" "chr1_134253" "chr1_134254"
[6] "chr1_134259"

[1] 144579

In [92]:
length(unique(setdiff(iso_no_IR_exon_region$isoform_id,unique(resInSpliceSites$V12))))

[1] 100265

In [93]:
length(unique(iso_no_IR_exon_region$isoform_id))

[1] 102950

In [ ]:

resInInIsoIR_region <- read.table("/disk1/wenqing/tmp_data/ASD/iso_no_IR_RES.txt")
dim(resInInIsoIR_region)

resNotInSpliceSites <- resInInIsoIR_region[!(resInInIsoIR_region$V1 %in% resInSpliceSites$V10),]
dim(resNotInSpliceSites)
length(unique(resNotInSpliceSites$V2))

In [56]:
length(unique(resInSpliceSites$V12))

[1] 1410

In [124]:
my_ka <- matrix(c(882,44004,4796,144579),nrow=2,byrow=T)
colnames(my_ka) <- c('0','1')

rownames(my_ka) <- c('0','1')
my_ka
chisq.test(my_ka)

,0,1
0,882,44004
1,4796,144579



	Pearson's Chi-squared test with Yates' continuity correction

data:  my_ka
X-squared = 188.33, df = 1, p-value < 2.2e-16


In [125]:
882/44004
4796/144579


[1] 0.02004363

[1] 0.03317218

In [78]:
2.993e-07 < 0.05

[1] TRUE